### TRATAMENTO E PREPARAÇÃO DOS DADOS PARA ANÁLISE EXPLORATÓRIA

**Por Isabella de Moura Rocha**

Graduada em Administração, pela UFSCar Sorocaba e Mestra em Administração, pela UFSCar Sorocaba, isabellamrocha0@gmail.com

*Os dados foram usados na dissertação: ROCHA, I.M. Rentabilidade, eficiência e risco em cooperativas de crédito e bancos comerciais. 2023. Dissertação (Mestrado em Administração) – Universidade Federal de São Carlos, Sorocaba, 2023. Disponível em: https://repositorio.ufscar.br/handle/ufscar/18956.*

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from copy import deepcopy
from sklearn import preprocessing
import requests

### 1. Importo os DFs necessários

In [2]:
df_banc=pd.read_parquet('/home/isabella/Masters/DATA/bancos_jun_dez_parq.parquet') # Balancetes consolidados Bancos
df_coop=pd.read_parquet('/home/isabella/Masters/DATA/cooperativas_jun_dez_parq.parquet') # Balancetes consolidados Cooperativas de crédito

In [3]:
cadastro=pd.read_parquet('/home/isabella/Masters/dados_novos/dados_cadastrais_cooperativas_bancos_comerciais.parquet') # Dados cadastrais derivados 

### 2. Tratamento dos dados

#### 2.1 Os saldos devem ser convertidos em valores numéricos

In [4]:
def arruma_df(valor): # arruma coluna 'SALDO'
    try:
        v = valor.replace(".", "")
        f = float(v)
    except:
        f = valor
    return f

In [5]:
df_banc['SALDO'] = df_banc['SALDO'].apply(arruma_df) # aplica função arruma saldo
df_coop['SALDO'] = df_coop['SALDO'].apply(arruma_df) # aplica função arruma saldo

#### 2.2 Tenho interesse apenas nos documentos 4010 e 4016 das entidades supervisionadas

Há contas de interesse que ora aparecem apenas no documento 4010, ora aparecem no 4016. 

Consultem documentação do BCB:

- Doc. 4010: https://www.bcb.gov.br/fis/info/doc4010.asp?frame=1
- Doc. 4016: https://www.bcb.gov.br/fis/info/doc4016.asp?frame=1 

In [6]:
df_banc = df_banc[df_banc['DOCUMENTO'].isin([4016, 4010])] # RECEITA E DESPESA aparecem apenas no doc. 4010 e o restante no 4016
df_coop = df_coop[df_coop['DOCUMENTO'].isin([4016, 4010])] # RECEITA E DESPESA aparecem apenas no doc. 4010 e o restante no 4016

Passo para datetime a coluna com DATA

In [7]:
df_banc['DATA'] = pd.to_datetime(df_banc['DATA'], format='%Y%m')
df_coop['DATA'] = pd.to_datetime(df_coop['DATA'], format='%Y%m')

#### 2.3 Vou comparar Bancos COMERCIAIS com Cooperativas de Crédito

Então filtro apenas aqueles bancos cuja taxonomia é relacionada aos bancos comerciais. 

Para cooperativas de crédito não faço o mesmo, pois quero comparar bancos comerciais com todas as cooperativas de crédito.

In [8]:
df_banc = df_banc[df_banc['TAXONOMIA'].isin(['BANCOS MULTIPLOS', 'BANCOS COMERCIAIS', 'BANCOS COMERCIAIS ESTRAGEIROS - FILIAL PAIS'])]

### 3. Cálculo do tamanho das contas de ATIVO de ambas as entidades

In [9]:
df_banc['CONTA'] = df_banc['CONTA'].astype(str) 
df_banc['PRIMEIRO_DIGITO'] = df_banc['CONTA'].str[0] 

df_coop['CONTA'] = df_coop['CONTA'].astype(str) 
df_coop['PRIMEIRO_DIGITO'] = df_coop['CONTA'].str[0] 

ativo_banco = df_banc[df_banc['PRIMEIRO_DIGITO'].isin(['1', '2', '3'])] # As contas que se iniciam com 1, 2 e 3 são de ativo
ativo_coope = df_coop[df_coop['PRIMEIRO_DIGITO'].isin(['1', '2', '3'])] # As contas que se iniciam com 1, 2 e 3 são de ativo

ativo_banco = ativo_banco.groupby(['DATA','DOCUMENTO','TAXONOMIA','CNPJ'])['SALDO'].sum().reset_index()
ativo_coope = ativo_coope.groupby(['DATA','DOCUMENTO','TAXONOMIA','CNPJ'])['SALDO'].sum().reset_index()

In [10]:
ativo_banco.rename(columns={'SALDO': 'ativo'}, inplace=True)
ativo_banco['ativo_ln'] = np.log(ativo_banco['ativo'])

ativo_coope.rename(columns={'SALDO': 'ativo'}, inplace=True)
ativo_coope['ativo_ln'] = np.log(ativo_coope['ativo'])

### 4. Cálculo do ROE e do ROA de ambas as entidades

*BITTENCOURT, W.R. et al, 2017. Rentabilidade em bancos múltiplos e cooperativas de crédito brasileiros. Revista de Administração Contemporânea, 21, pp.22-40.*

#### 4.1 Filtro as contas contábeis COSIF de interesse

In [11]:
contas = ['70000009','80000006','60000002','10000007','20000004'] # detalhes no artigo base da seção

df_banc = df_banc[df_banc['CONTA'].isin(contas)]
df_coop = df_coop[df_coop['CONTA'].isin(contas)]

#### 4.2 Pivoto para conseguir trabalhar mais facilmente

In [12]:
pivot_banc = df_banc.pivot_table(index=['DATA','DOCUMENTO','TAXONOMIA','CNPJ'], columns='CONTA', values='SALDO').reset_index()
pivot_coop = df_coop.pivot_table(index=['DATA','DOCUMENTO','TAXONOMIA','CNPJ'], columns='CONTA', values='SALDO').reset_index()

#### 4.3 Calculo o ROE e o ROA para cada entidade

In [13]:
pivot_banc['roe'] = (pivot_banc['70000009'] + pivot_banc['80000006']) / pivot_banc['60000002'] # detalhes no artigo base da seção
pivot_banc['roa'] = (pivot_banc['70000009'] + pivot_banc['80000006']) / (pivot_banc['10000007'] + pivot_banc['20000004']) # detalhes no artigo base da seção

In [17]:
completo_banco = pd.merge(pivot_banc, ativo_banco, on=['DATA', 'DOCUMENTO', 'TAXONOMIA', 'CNPJ'], how='left')
completo_coope = pd.merge(pivot_coop, ativo_coope, on=['DATA', 'DOCUMENTO', 'TAXONOMIA', 'CNPJ'], how='left')

In [21]:
completo_banco['ano'] = completo_banco['DATA'].dt.year # quero criar uma coluna ano
completo_coope['ano'] = completo_coope['DATA'].dt.year # quero criar uma coluna ano 

In [22]:
completo_banco.to_parquet('/home/isabella/Masters/dados_novos/ativo_rentabilidade_banco.parquet')

In [23]:
completo_coope.to_parquet('/home/isabella/Masters/dados_novos/ativo_rentabilidade_coop.parquet')